# Home Depot Product Search Relevance IRDM Porject

## <font color='green'>Setup 1</font>: Load Libraries and Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
start_time = time.time()

import numpy as np
import pandas as pd
import re
import os

from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

stemmer = SnowballStemmer('english')

In [2]:
from config import *
from homedepot_functions_and_dictionary.py import *

In [3]:
stoplist = stopwords.words('english')
stoplist.append('till') 
stoplist_wo_can=stoplist[:]
stoplist_wo_can.remove('can')

In [4]:
df_train = pd.read_csv(DATA_DIR+'/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(DATA_DIR+'/test.csv', encoding="ISO-8859-1")
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

df_attr = pd.read_csv(DATA_DIR+'/attributes.csv', encoding="ISO-8859-1")
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})

df_all = pd.merge(df_all, df_attr[df_attr['name']=="MFG Brand Name"][['product_uid','value']], how='left', on='product_uid')
df_all['brand']=df_all['value'].fillna("").map(lambda x: x.encode('utf-8'))
df_all=df_all.drop('value',axis=1)

In [5]:
num_train = df_train.shape[0]
num_test = df_test.shape[0]

In [6]:
df_all[:3]

,id,product_title,product_uid,relevance,search_term,brand
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.0,angle bracket,Simpson Strong-Tie
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.5,l bracket,Simpson Strong-Tie
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.0,deck over,BEHR Premium Textured DeckOver


## <font color='green'>Setup 2</font>: Data Processing

### Part 1: Preparation
Create a list of words with lowercase and uppercase letters (after simple parsing) from brand and product title, eg: DeckOver

In [7]:
add_space_stop_list=[]
uniq_brands=list(set(list(df_all['brand'])))
for i in range(0,len(uniq_brands)):
    uniq_brands[i]=simple_parser(uniq_brands[i])               # simple parser
    if re.search(r'[a-z][A-Z][a-z]',uniq_brands[i])!=None:     # word with a captial letter within eg: DeckOver---kOv
        for word in uniq_brands[i].split():
            if re.search(r'[a-z][A-Z][a-z]',word)!=None:
                add_space_stop_list.append(word.lower())
add_space_stop_list=list(set(add_space_stop_list))      
print len(add_space_stop_list)," words from brands in add_space_stop_list"
                
uniq_titles=list(set(list(df_all['product_title'])))
for i in range(0,len(uniq_titles)):
    uniq_titles[i]=simple_parser(uniq_titles[i])
    if re.search(r'[a-z][A-Z][a-z]',uniq_titles[i])!=None:
        for word in uniq_titles[i].split():
            if re.search(r'[a-z][A-Z][a-z]',word)!=None:
                add_space_stop_list.append(word.lower())    
add_space_stop_list=list(set(add_space_stop_list))      

458  words from brands in add_space_stop_list


### Part 2: Spell Check

Do spell check on search term using the Google dict from the forum https://www.kaggle.com/steubk/home-depot-product-search-relevance/fixing-typos


In [8]:
df_all['search_term']=df_all['search_term'].map(lambda x: google_dict[x] if x in google_dict.keys() else x)   

* construct the bigrams of concatenated product titles 
* count word frequencies for search term and product title, save as a dictionary

In [9]:
def is_word_in_string(word,s):
    return word in s.split() 
    
def create_bigrams(s):
    lst=[word for word in s.split() if len(re.sub('[^0-9]', '', word))==0 and len(word)>2]
    output=""
    i=0
    if len(lst)>=2:
        while i<len(lst)-1:
            output+= " "+lst[i]+"_"+lst[i+1]
            i+=1
    return output


df_all['product_title_simpleparsed']=df_all['product_title'].map(lambda x: simple_parser(x).lower())
df_all['search_term_simpleparsed']=df_all['search_term'].map(lambda x: simple_parser(x).lower())


# concat all titles/queries into a string 
str_title=" ".join(list(df_all['product_title'].map(lambda x: simple_parser(x).lower())))
str_query=" ".join(list(df_all['search_term'].map(lambda x: simple_parser(x).lower())))

# create bigrams of prodict title
bigrams_str_title=" ".join(list(df_all['product_title'].map(lambda x: create_bigrams(simple_parser(x).lower()))))
bigrams_set=set(bigrams_str_title.split())

### count word frequencies for query and product title
my_dict={}
str1= str_title+" "+str_query
for word in list(set(list(str1.split()))):
    my_dict[word]={"title":0, "query":0, 'word':word}
for word in str_title.split():
    my_dict[word]["title"]+=1    
for word in str_query.split():
    my_dict[word]["query"]+=1

Process words in frequncy dictionary without digits.

For **word that appears only in query**, if word length >3/ no numbers in word, add word into error dictionary unless:

* word has meaning (according to wn.synsets)
* singular/pular form of word exists in word frequency dictionary

Assume the corresponding correct words of error words exist if words appers 5 or more times in product title

In [10]:
errors_dict={}
correct_dict={}
for word in my_dict.keys():
    if len(word)>=3 and len(re.sub('[^0-9]', '', word))==0:
        if my_dict[word]["title"]==0:
            if len(wn.synsets(word))>0 \
            or (word.endswith('s') and  (word[:-1] in my_dict.keys()) and my_dict[word[:-1]]["title"]>0)\
            or (word[-1]!='s' and (word+'s' in my_dict.keys()) and my_dict[word+'s']["title"]>0):
                1
            else:
                errors_dict[word]=my_dict[word]
        elif my_dict[word]["title"]>=5:
            correct_dict[word]=my_dict[word]

For each error word try finding a good match in bigrams, matched products, all products, with piority: bigram > mathced prodcuts > all products 

* (suggestion is) bigram: if the word matches with elements in bigram list after removing '-' (eg. air-conditioner --> aircondictioner)
* (suggestion is) mathced prodcuts: for all search terms after simpleparsing where the word exists in, search all words in the corresponding product title, if its relavent index excesses a certain threshold, keep it in a tuple, Finally select the word with maximum index. 

Save as automatically_generated_word_corrections.csv in preprocessing folder

In [11]:
cnt=0
NN=len(errors_dict.keys())

for i in range(0,len(errors_dict.keys())):
    word=sorted(errors_dict.keys())[i]
    cnt+=1
    lst=[]
    lst_tuple=[]
    suggested=False
    suggested_word=""
    rt_max=0
    
    # if only one word in query, use be more selective in choosing a correction
    min_query_len=min(df_all['search_term_simpleparsed'][df_all['search_term_simpleparsed'].map(lambda x: is_word_in_string(word,x))].map(lambda x: len(x.split())))
    delta=0.05*int(min_query_len<2)
    
    words_from_matched_titles=[item for item in \
        " ".join(list(set(df_all['product_title_simpleparsed'][df_all['search_term_simpleparsed'].map(lambda x: is_word_in_string(word,x))]))).split() \
        if len(item)>2 and len(re.sub('[^0-9]', '', item))==0]
    words_from_matched_titles=list(set(words_from_matched_titles))
    words_from_matched_titles.sort()
    
    source=""
    for bigram in bigrams_set:
        if bigram.replace("_","")==word:
            suggested=True
            suggested_word=bigram.replace("_"," ")
            source="from bigrams"
            
    if source=="":
        for correct_word in words_from_matched_titles: 
            rt, rt_scaled = seq_matcher(word,correct_word)
            
            if rt>0.75+delta or (len(word)<6 and rt>0.68+delta):
                lst.append(correct_word)
                lst_tuple.append((correct_word,my_dict[correct_word]["title"]))
                if rt>rt_max:
                    rt_max=rt
                    suggested=True
                    source="from matched products"
                    suggested_word=correct_word
                elif rt==rt_max and seq_matcher("".join(sorted(word)),"".join(sorted(correct_word)))[0]>seq_matcher("".join(sorted(word)),"".join(sorted(suggested_word)))[0]:
                    suggested_word=correct_word
                elif rt==rt_max:
                    suggested=False
                    source=""
        
    if source=="" and len(lst)==0:
        source="from all products"
        for correct_word in correct_dict.keys():
            rt, rt_scaled = seq_matcher(word,correct_word)
            #print correct_word, rt,rt_scaled
            if correct_dict[correct_word]["title"]>10 and (rt>0.8+delta or (len(word)<6 and rt>0.73+delta)):
                #print correct_word, rt,rt_scaled
                lst.append(correct_word)
                lst_tuple.append((correct_word,correct_dict[correct_word]["title"]))
                if rt>rt_max:
                    rt_max=rt
                    suggested=True
                    suggested_word=correct_word
                elif rt==rt_max and seq_matcher("".join(sorted(word)),"".join(sorted(correct_word)))[0]>seq_matcher("".join(sorted(word)),"".join(sorted(suggested_word)))[0]:
                    suggested_word=correct_word
                elif rt==rt_max: 
                    suggested=False

    if suggested==True:
        errors_dict[word]["suggestion"]=suggested_word
        errors_dict[word]["others"]=lst_tuple
        errors_dict[word]["source"]=source
    else:
        errors_dict[word]["suggestion"]=""
        errors_dict[word]["others"]=lst_tuple
        errors_dict[word]["source"]=source


### Add some words with digits
### If the word begins with a meanigful part [len(wn.synsets(srch.group(0)))>0],
### ends with a number and has vowels
for word in my_dict.keys():
    if my_dict[word]['query']>0 and my_dict[word]['title']==0 \
    and len(re.sub('[^0-9]', '', word))!=0 and len(re.sub('[^a-z]', '', word))!=0:
        srch=re.search(r'(?<=^)[a-z][a-z][a-z]+(?=[0-9])',word)
        if srch!=None and len(wn.synsets(srch.group(0)))>0 \
        and len(re.sub('[^aeiou]', '', word))>0 and word[-1] in '0123456789': 
            errors_dict[word]=my_dict[word]
            errors_dict[word]["source"]="added space before digit"
            errors_dict[word]["suggestion"]=re.sub(r'(?<=^)'+srch.group(0)+r'(?=[a-zA-Z0-9])',srch.group(0)+' ',word)

### save dictionary
corrections_df=pd.DataFrame(errors_dict).transpose()
corrections_df.to_csv(PROCESSINGTEXT_DIR+"/automatically_generated_word_corrections.csv")

In [12]:
t0=time()
spell_check_dict={}
for word in errors_dict.keys():
    if errors_dict[word]['suggestion']!="":
        spell_check_dict[word]=errors_dict[word]['suggestion']
        
### parse query and product title
df_all['search_term_parsed']=col_parser(df_all['search_term'],automatic_spell_check_dict=spell_check_dict,\
            add_space_stop_list=[]).map(lambda x: x.encode('utf-8'))
df_all['search_term_parsed_wospellcheck']=col_parser(df_all['search_term'],automatic_spell_check_dict={},\
            add_space_stop_list=[]).map(lambda x: x.encode('utf-8'))    

parsed 0 out of 24469 unique values; 0.0 minutes
parsed 10000 out of 24469 unique values; 8.2 minutes
parsed 20000 out of 24469 unique values; 16.3 minutes
parsed 0 out of 24469 unique values; 0.0 minutes
parsed 10000 out of 24469 unique values; 1.8 minutes
parsed 20000 out of 24469 unique values; 3.6 minutes


Compare search_term_simpleparsed and search_term_parsed

In [13]:
import random

for i,j in df_all.iterrows():
    if random.choice(spell_check_dict.keys()) in j['search_term_simpleparsed']:
        break 
    
df_all[i:i+1][['search_term_simpleparsed','search_term_parsed']]

,search_term_simpleparsed,search_term_parsed
119,kohler rosario toilet parts,kohler rosario toilet parts


In [14]:
t0 = time()

### function to check whether queries parsed with and without spell correction are identical
def match_queries(q1,q2):
    q1=re.sub('[^a-z\ ]', '', q1)
    q2=re.sub('[^a-z\ ]', '', q2)
    q1= " ".join([word[0:(len(word)-int(word[-1]=='s'))] for word in q1.split()])
    q2= " ".join([word[0:(len(word)-int(word[-1]=='s'))] for word in q2.split()])
    return difflib.SequenceMatcher(None, q1,q2).ratio()


df_all['is_query_misspelled']=df_all.apply(lambda x: \
            match_queries(x['search_term_parsed'],x['search_term_parsed_wospellcheck']),axis=1)
df_all=df_all.drop(['search_term_parsed_wospellcheck'],axis=1)    
print 'create dummy "is_query_misspelled" time:',round((time()-t0)/60,1) ,'minutes\n'


t0 = time()
df_all['product_title_parsed']=col_parser(df_all['product_title'],add_space_stop_list=[],\
                remove_from_brackets=True).map(lambda x: x.encode('utf-8'))
print 'product_title parsing time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()


create dummy "is_query_misspelled" time: 0.3 minutes

parsed 0 out of 120453 unique values; 0.0 minutes
parsed 10000 out of 120453 unique values; 1.8 minutes
parsed 20000 out of 120453 unique values; 3.7 minutes
parsed 30000 out of 120453 unique values; 5.5 minutes
parsed 40000 out of 120453 unique values; 7.4 minutes
parsed 50000 out of 120453 unique values; 9.2 minutes
parsed 60000 out of 120453 unique values; 11.1 minutes
parsed 70000 out of 120453 unique values; 13.1 minutes
parsed 80000 out of 120453 unique values; 15.0 minutes
parsed 90000 out of 120453 unique values; 16.9 minutes
parsed 100000 out of 120453 unique values; 18.7 minutes
parsed 110000 out of 120453 unique values; 20.5 minutes
parsed 120000 out of 120453 unique values; 22.4 minutes
product_title parsing time: 22.5 minutes



### Part 3: Brand  Name Modification 

Some brand names in "MFG Brand Name" of attributes.csv are consisted by a few words, but it is much more likely for a person to search for brand 'BEHR' than 'BEHR PREMIUM PLUS ULTRA'. Therefore here we replace long brand names with a (hand-created) shorter alternatives 

In [15]:
replace_brand_dict={
'acurio latticeworks': 'acurio', 'american kennel club':'akc','amerimax home products': 'amerimax',
'barclay products':'barclay','behr marquee': 'behr', 'behr premium': 'behr', 'behr premium deckover': 'behr', 
'behr premium plus': 'behr', 'behr premium plus ultra': 'behr', 'behr premium textured deckover': 'behr', 
'behr pro': 'behr', 'bel air lighting': 'bel air','bootz industries':'bootz','campbell hausfeld':'campbell',
'columbia forest products': 'columbia','essick air products':'essick air','evergreen enterprises':'evergreen',
'feather river doors': 'feather river', 'gardner bender':'gardner','ge parts':'ge','ge reveal':'ge',
'gibraltar building products':'gibraltar','gibraltar mailboxes':'gibraltar','glacier bay':'glacier',
'great outdoors by minka lavery': 'great outdoors', 'hamilton beach': 'hamilton','hampton bay':'hampton',
'hampton bay quickship':'hampton','handy home products':'handy home','hickory hardware': 'hickory', 
'home accents holiday': 'home accents','home decorators collection': 'home decorators','homewerks worldwide':'homewerks',
'klein tools': 'klein','lakewood cabinets':'lakewood','leatherman tool group':'leatherman','legrand adorne':'legrand',
'legrand wiremold':'legrand','lg hausys hi macs':'lg','lg hausys viatera':'lg','liberty foundry':'liberty',
'liberty garden':'liberty','lithonia lighting':'lithonia','loloi rugs':'loloi','maasdam powr lift':'maasdam',
'maasdam powr pull':'maasdam','martha stewart living': 'martha stewart','merola tile': 'merola','miracle gro':'miracle',
'miracle sealants':'miracle','mohawk home': 'mohawk','mtd genuine factory parts':'mtd','mueller streamline': 'mueller',
'newport coastal': 'newport','nourison overstock':'nourison','nourison rug boutique':'nourison','owens corning': 'owens', 
'premier copper products':'premier','price pfister':'pfister','pride garden products':'pride garden','prime line products':'prime line',
'redi base':'redi','redi drain':'redi','redi flash':'redi','redi ledge':'redi','redi neo':'redi','redi niche':'redi','redi shade':'redi',
'redi trench':'redi','reese towpower':'reese','rheem performance': 'rheem','rheem ecosense': 'rheem','rheem performance plus': 'rheem',
'rheem protech': 'rheem','richelieu hardware':'richelieu','rubbermaid commercial products': 'rubbermaid', 
'rust oleum american accents': 'rust oleum', 'rust oleum automotive': 'rust oleum', 'rust oleum concrete stain': 'rust oleum', 
'rust oleum epoxyshield': 'rust oleum', 'rust oleum flexidip': 'rust oleum', 'rust oleum marine': 'rust oleum', 
'rust oleum neverwet': 'rust oleum', 'rust oleum parks': 'rust oleum', 'rust oleum professional': 'rust oleum', 
'rust oleum restore': 'rust oleum', 'rust oleum rocksolid': 'rust oleum', 'rust oleum specialty': 'rust oleum', 
'rust oleum stops rust': 'rust oleum', 'rust oleum transformations': 'rust oleum', 'rust oleum universal': 'rust oleum', 
'rust oleum painter touch 2': 'rust oleum','rust oleum industrial choice':'rust oleum','rust oleum okon':'rust oleum',
'rust oleum painter touch':'rust oleum','rust oleum painter touch 2':'rust oleum','rust oleum porch and floor':'rust oleum',
'salsbury industries':'salsbury','simpson strong tie': 'simpson', 'speedi boot': 'speedi', 'speedi collar': 'speedi', 
'speedi grille': 'speedi', 'speedi products': 'speedi', 'speedi vent': 'speedi', 'pass and seymour': 'seymour',
'pavestone rumblestone': 'rumblestone','philips advance':'philips','philips fastener':'philips','philips ii plus':'philips',
'philips manufacturing company':'philips','safety first':'safety 1st','sea gull lighting': 'sea gull','scott':'scotts',
'scotts earthgro':'scotts','south shore furniture': 'south shore', 'tafco windows': 'tafco','trafficmaster allure': 'trafficmaster', 
'trafficmaster allure plus': 'trafficmaster', 'trafficmaster allure ultra': 'trafficmaster', 'trafficmaster ceramica': 'trafficmaster', 
'trafficmaster interlock': 'trafficmaster', 'thomas lighting': 'thomas', 'unique home designs':'unique home','veranda hp':'veranda',
'whitehaus collection':'whitehaus','woodgrain distritubtion':'woodgrain','woodgrain millwork': 'woodgrain', 
'woodford manufacturing company': 'woodford', 'wyndham collection':'wyndham','yardgard select': 'yardgard','yosemite home decor': 'yosemite'
}

df_all['brand_parsed']=col_parser(df_all['brand'].map(lambda x: re.sub('^[t|T]he ', '', x.replace(".N/A","").replace("N.A.","").replace("n/a","").replace("Generic Unbranded","").replace("Unbranded","").replace("Generic",""))),add_space_stop_list=add_space_stop_list)
list_brands=list(df_all['brand_parsed'])

df_all['brand_parsed']=df_all['brand_parsed'].map(lambda x: replace_brand_dict[x] if x in replace_brand_dict.keys() else x)


parsed 0 out of 4284 unique values; 0.0 minutes


Create a brand dictionary which stores the count frequencies of brands search term and product title
* some words in brand names such as 'design' might have other meanings, thus we create a list to delte these word.
* only include tile which exists either 8+ times in product title or [1+ time in query and 3+ times in product title]

In [16]:
str_query=" : ".join(list(df_all['search_term_parsed'])).lower()
print "\nGenerating brand dict: How many times each brand appears in the dataset?"
t0 = time()
brand_dict=get_attribute_dict(list_brands,str_query=str_query)

### These words are likely to mean other things than brand names. 
### For example, it would not be prudent to consider each occurence of 'design' or 'veranda' as a brand name.
### We decide not to use these words as brands and exclude them from our brand dictionary.
# The list is shared on the forum.
del_list=['aaa','off','impact','square','shelves','finish','ring','flood','dual','ball','cutter',\
'max','off','mat','allure','diamond','drive', 'edge','anchor','walls','universal','cat', 'dawn','ion','daylight',\
'roman', 'weed eater', 'restore', 'design', 'caddy', 'pole caddy', 'jet', 'classic', 'element', 'aqua',\
'terra', 'decora', 'ez', 'briggs', 'wedge', 'sunbrella',  'adorne', 'santa', 'bella', 'duck', 'hotpoint',\
'duck', 'tech', 'titan', 'powerwasher', 'cooper lighting', 'heritage', 'imperial', 'monster', 'peak', 
'bell', 'drive', 'trademark', 'toto', 'champion', 'shop vac', 'lava', 'jet', 'flood', \
'roman', 'duck', 'magic', 'allen', 'bunn', 'element', 'international', 'larson', 'tiki', 'titan', \
 'space saver', 'cutter', 'scotch', 'adorne', 'ball', 'sunbeam', 'fatmax', 'poulan', 'ring', 'sparkle', 'bissell', \
 'universal', 'paw', 'wedge', 'restore', 'daylight', 'edge', 'americana', 'wacker', 'cat', 'allure', 'bonnie plants', \
 'troy', 'impact', 'buffalo', 'adams', 'jasco', 'rapid dry', 'aaa', 'pole caddy', 'pac', 'seymour', 'mobil', \
 'mastercool', 'coca cola', 'timberline', 'classic', 'caddy', 'sentry', 'terrain', 'nautilus', 'precision', \
 'artisan', 'mural', 'game', 'royal', 'use', 'dawn', 'task', 'american line', 'sawtrax', 'solo', 'elements', \
 'summit', 'anchor', 'off', 'spruce', 'medina', 'shoulder dolly', 'brentwood', 'alex', 'wilkins', 'natural magic', \
 'kodiak', 'metro', 'shelter', 'centipede', 'imperial', 'cooper lighting', 'exide', 'bella', 'ez', 'decora', \
 'terra', 'design', 'diamond', 'mat', 'finish', 'tilex', 'rhino', 'crock pot', 'legend', 'leatherman', 'remove', \
 'architect series', 'greased lightning', 'castle', 'spirit', 'corian', 'peak', 'monster', 'heritage', 'powerwasher',\
 'reese', 'tech', 'santa', 'briggs', 'aqua', 'weed eater', 'ion', 'walls', 'max', 'dual', 'shelves', 'square',\
 'hickory', "vikrell", "e3", "pro series", "keeper", "coastal shower doors", 'cadet','church','gerber','glidden',\
 'cooper wiring devices', 'border blocks', 'commercial electric', 'pri','exteria','extreme', 'veranda',\
 'gorilla glue','gorilla','shark','wen']
del_list=list(set(list(del_list)))

for key in del_list:
    if key in brand_dict.keys():
        del(brand_dict[key])

# save to file
brand_df=pd.DataFrame(brand_dict).transpose()
brand_df.to_csv(PROCESSINGTEXT_DIR+"/brand_statistics.csv")

for item in brand_dict.keys():
    if (brand_dict[item]['cnt_attribute']>=3 and brand_dict[item]['cnt_query']>=1) \
    or (brand_dict[item]['cnt_attribute'])>=8:
        1
    else:
        del(brand_dict[item])

brand_df=pd.DataFrame(brand_dict).transpose().sort(['cnt_query'], ascending=[1])
print 'brand dict creation time:',round((time()-t0)/60,1) ,'minutes\n'


Generating brand dict: How many times each brand appears in the dataset?
('500 out of 4266 unique attributes', 0.7, 'minutes')
('1000 out of 4266 unique attributes', 1.5, 'minutes')
('1500 out of 4266 unique attributes', 2.2, 'minutes')
('2000 out of 4266 unique attributes', 2.9, 'minutes')
('2500 out of 4266 unique attributes', 3.7, 'minutes')
('3000 out of 4266 unique attributes', 4.4, 'minutes')
('3500 out of 4266 unique attributes', 5.1, 'minutes')
('4000 out of 4266 unique attributes', 5.8, 'minutes')
brand dict creation time: 6.2 minutes



### Part 4: Materials

Create list of unique materials for each product_uid from attribution csv. For some product_uid contains multiple different values of "Material", thus we have to concatenate all such values to ensure that each product_uid has only one value for material.

In addition a replace dictionary is created in order to shorten some material name.

In [17]:
tmp_material=df_attr[df_attr['name']=="Material"][['product_uid','value']]
tmp_material=tmp_material[tmp_material['value']!="Other"]
tmp_material=tmp_material[tmp_material['value']!="*"]
def change_material(s):
    replace_dict={'Medium Density Fiberboard (MDF)':'mdf', 'High Density Fiberboard (HDF)':'hdf',\
    'Fibre Reinforced Polymer (FRP)': 'frp', 'Acrylonitrile Butadiene Styrene (ABS)': 'abs',\
    'Cross-Linked Polyethylene (PEX)':'pex', 'Chlorinated Poly Vinyl Chloride (CPVC)': 'cpvc',\
    'PVC (vinyl)': 'pvc','Thermoplastic rubber (TPR)':'tpr','Poly Lactic Acid (PLA)': 'pla',\
    '100% Polyester':'polyester','100% UV Olefin':'olefin', '100% BCF Polypropylene': 'polypropylene',\
    '100% PVC':'pvc'}
        
    if s in replace_dict.keys():
        s=replace_dict[s]
    return s
    
tmp_material['value'] = tmp_material['value'].map(lambda x: change_material(x))

create material dictionary which capture the count frequencies of materials in search term and attribute

In [18]:
dict_materials = {}
key_list=tmp_material['product_uid'].keys()
for i in range(0,len(key_list)):
    if tmp_material['product_uid'][key_list[i]] not in dict_materials.keys():
        dict_materials[tmp_material['product_uid'][key_list[i]]]={}
        dict_materials[tmp_material['product_uid'][key_list[i]]]['product_uid']=tmp_material['product_uid'][key_list[i]]
        dict_materials[tmp_material['product_uid'][key_list[i]]]['cnt']=1
        dict_materials[tmp_material['product_uid'][key_list[i]]]['material']=tmp_material['value'][key_list[i]]
    else:
        ##print key_list[i]
        dict_materials[tmp_material['product_uid'][key_list[i]]]['material']=dict_materials[tmp_material['product_uid'][key_list[i]]]['material']+' '+tmp_material['value'][key_list[i]]
        dict_materials[tmp_material['product_uid'][key_list[i]]]['cnt']+=1
    if (i % 10000)==0:
        print i
                       
df_materials=pd.DataFrame(dict_materials).transpose()

### merge created 'material' column with df_all
df_all = pd.merge(df_all, df_materials[['product_uid','material']], how='left', on='product_uid')
df_all['material']=df_all['material'].fillna("").map(lambda x: x.encode('utf-8'))

df_all['material_parsed']=col_parser(df_all['material'].map(lambda x: x.replace("Other","").replace("*","")), parse_material=True,add_space_stop_list=[])

### list of all materials
list_materials=list(df_all['material_parsed'].map(lambda x: x.lower())) 

### count frequencies of materials in query and product_title
print "\nGenerating material dict: How many times each material appears in the dataset?"
material_dict=get_attribute_dict(list_materials,str_query=str_query)

### create dataframe and save to file
material_df=pd.DataFrame(material_dict).transpose()
material_df.to_csv(PROCESSINGTEXT_DIR+"/material_statistics.csv")


### For further processing keep only materials that appear 
### more 10+ times in product_title and at least once in query
for key in set(material_dict.keys()):
    if material_dict[key]['cnt_attribute']<20 or material_dict[key]['cnt_query']>3*material_dict[key]['cnt_attribute']:
        del(material_dict[key])

material_df=pd.DataFrame(material_dict).transpose().sort(['cnt_query'], ascending=[1])
print 'material dict creation time:',round((time()-t0)/60,1) ,'minutes\n'

0
10000
20000
30000
parsed 0 out of 856 unique values; 0.0 minutes

Generating material dict: How many times each material appears in the dataset?
('500 out of 777 unique attributes', 0.8, 'minutes')
material dict creation time: 7.9 minutes



In [19]:
material_df

,cnt_attribute,cnt_query,name,nwords
manufactured from laminated particle board,32,0,manufactured from laminated particle board,5
hdf,114,0,hdf,1
hdf laminate,29,0,hdf laminate,2
hdpe. high density polethylene.,46,0,hdpe. high density polethylene.,4
iron ferrite,23,0,iron ferrite,2
laminate hdf,72,0,laminate hdf,2
laminate laminate. high quality hdf,28,0,laminate laminate. high quality hdf,5
laminate mdf. paper. resin. both acrylic and melamine. polyurethane hot metal adhesive and olefin hot metal adhesive,132,0,laminate mdf. paper. resin. both acrylic and m...,17
laminate mdf. paper. resin. both acrylic and melamine. polyurethane hot metal and olefin hot metal adhesive,68,0,laminate mdf. paper. resin. both acrylic and m...,16
laminated particle board,111,0,laminated particle board,3


In [20]:
def getremove_brand_or_material_from_str(s,df, replace_brand_dict={}):
    items_found=[]
    df=df.sort_values(['nwords'],ascending=[0])
    key_list=df['nwords'].keys()
    #start with several-word brands or materials
    #assert df['nwords'][key_list[0]]>1
    for i in range(0,len(key_list)):
        item=df['name'][key_list[i]]
        if item in s:
            if re.search(r'\b'+item+r'\b',s)!=None:
                s=re.sub(r'\b'+item+r'\b', '', s)
                if item in replace_brand_dict.keys():
                    items_found.append(replace_brand_dict[item])
                else:
                    items_found.append(item)

    return " ".join(s.split()), ";".join(items_found)


aa=list(set(list(df_all['search_term_parsed'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=getremove_brand_or_material_from_str(aa[i],brand_df)
    if (i % 5000)==0:
        print "Extracted brands from",i,"out of",len(aa),"unique search terms; ", str(round((time()-t0)/60,1)),"minutes"
df_all['search_term_tuple']= df_all['search_term_parsed'].map(lambda x: my_dict[x])
df_all['search_term_parsed_woBrand']= df_all['search_term_tuple'].map(lambda x: x[0])
df_all['brands_in_search_term']= df_all['search_term_tuple'].map(lambda x: x[1])
print 'extract brands from query time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

df_all['search_term_tuple']= df_all['search_term_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df))
df_all['search_term_parsed_woBM']= df_all['search_term_tuple'].map(lambda x: x[0])
df_all['materials_in_search_term']= df_all['search_term_tuple'].map(lambda x: x[1])
df_all=df_all.drop('search_term_tuple',axis=1)
print 'extract materials from query time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()


##############################
aa=list(set(list(df_all['product_title_parsed'])))
my_dict={}
for i in range(0,len(aa)):
    my_dict[aa[i]]=getremove_brand_or_material_from_str(aa[i],brand_df)
    if (i % 5000)==0:
        print "Extracted brands from",i,"out of",len(aa),"unique product titles; ", str(round((time()-t0)/60,1)),"minutes"

df_all['product_title_tuple']= df_all['product_title_parsed'].map(lambda x: my_dict[x])
df_all['product_title_parsed_woBrand']= df_all['product_title_tuple'].map(lambda x: x[0])
df_all['brands_in_product_title']= df_all['product_title_tuple'].map(lambda x: x[1])
print 'extract brands from product title time:',round((time()-t0)/60,1) ,'minutes\n'

df_all['product_title_tuple']= df_all['product_title_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df))
df_all['product_title_parsed_woBM']= df_all['product_title_tuple'].map(lambda x: x[0])
df_all['materials_in_product_title']= df_all['product_title_tuple'].map(lambda x: x[1])
df_all=df_all.drop('product_title_tuple',axis=1)

Extracted brands from 0 out of 24350 unique search terms;  7.9 minutes
Extracted brands from 5000 out of 24350 unique search terms;  10.3 minutes
Extracted brands from 10000 out of 24350 unique search terms;  12.6 minutes
Extracted brands from 15000 out of 24350 unique search terms;  14.9 minutes
Extracted brands from 20000 out of 24350 unique search terms;  17.2 minutes
extract brands from query time: 19.3 minutes

extract materials from query time: 6.0 minutes

Extracted brands from 0 out of 118401 unique product titles;  0.0 minutes
Extracted brands from 5000 out of 118401 unique product titles;  2.3 minutes
Extracted brands from 10000 out of 118401 unique product titles;  4.7 minutes
Extracted brands from 15000 out of 118401 unique product titles;  7.0 minutes
Extracted brands from 20000 out of 118401 unique product titles;  9.4 minutes
Extracted brands from 25000 out of 118401 unique product titles;  11.7 minutes
Extracted brands from 30000 out of 118401 unique product titles;  14

### Part 5: POS

use nltk.pos_tagger() to tag words

In [21]:
df_all['search_term_tokens'] =col_tagger(df_all['search_term_parsed_woBM'])
df_all['product_title_tokens'] =col_tagger(df_all['product_title_parsed_woBM'])

tagged 20000 out of 240760 total rows; 0.1 minutes
tagged 40000 out of 240760 total rows; 0.2 minutes
tagged 60000 out of 240760 total rows; 0.3 minutes
tagged 80000 out of 240760 total rows; 0.4 minutes
tagged 100000 out of 240760 total rows; 0.5 minutes
tagged 120000 out of 240760 total rows; 0.5 minutes
tagged 140000 out of 240760 total rows; 0.6 minutes
tagged 160000 out of 240760 total rows; 0.7 minutes
tagged 180000 out of 240760 total rows; 0.8 minutes
tagged 200000 out of 240760 total rows; 0.9 minutes
tagged 220000 out of 240760 total rows; 1.0 minutes
tagged 240000 out of 240760 total rows; 1.1 minutes
tagged 240760 out of 240760 total rows; 1.1 minutes
tagged 20000 out of 240760 total rows; 0.3 minutes
tagged 40000 out of 240760 total rows; 0.6 minutes
tagged 60000 out of 240760 total rows; 0.8 minutes
tagged 80000 out of 240760 total rows; 1.1 minutes
tagged 100000 out of 240760 total rows; 1.4 minutes
tagged 120000 out of 240760 total rows; 1.7 minutes
tagged 140000 out of

In [22]:
df_all

,id,product_title,product_uid,relevance,search_term,brand,product_title_simpleparsed,search_term_simpleparsed,search_term_parsed,is_query_misspelled,...,search_term_parsed_woBrand,brands_in_search_term,search_term_parsed_woBM,materials_in_search_term,product_title_parsed_woBrand,brands_in_product_title,product_title_parsed_woBM,materials_in_product_title,search_term_tokens,product_title_tokens
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,Simpson Strong-Tie,simpson strong tie 12 gauge angle,angle bracket,angle bracket,1.000000,...,angle bracket,,angle bracket,,12 gauge angle,simpson strong tie,12 gauge angle,,"[(angle, NN), (bracket, NN)]","[(12, CD), (gauge, JJ), (angle, NN)]"
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,Simpson Strong-Tie,simpson strong tie 12 gauge angle,l bracket,l bracket,1.000000,...,l bracket,,l bracket,,12 gauge angle,simpson strong tie,12 gauge angle,,"[(l, NN), (bracket, NN)]","[(12, CD), (gauge, NN), (angle, NN)]"
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DeckOver,behr premium textured deckover 1 gal sc 141 tu...,deck over,deckover,0.941176,...,deckover,,deckover,,1-gal sc141 tugboat wood and concrete coating,behr premium textured deckover,1-gal sc141 tugboat and coating,concrete;wood,"[(deckover, NN)]","[(1-gal, JJ), (sc141, NN), (tugboat, NN), (and..."
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Delta,delta vero 1 handle shower only faucet trim ki...,rain shower head,rain showerhead,1.000000,...,rain showerhead,,rain showerhead,,vero 1 handle shower only faucet trim kit in c...,delta,vero 1 handle shower only faucet trim kit in c...,,"[(rain, NN), (showerhead, NN)]","[(vero, NN), (1, CD), (handle, NN), (shower, J..."
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Delta,delta vero 1 handle shower only faucet trim ki...,shower only faucet,shower only faucet,1.000000,...,shower only faucet,,shower only faucet,,vero 1 handle shower only faucet trim kit in c...,delta,vero 1 handle shower only faucet trim kit in c...,,"[(shower, NN), (only, RB), (faucet, VBZ)]","[(vero, NN), (1, CD), (handle, NN), (shower, J..."
5,18,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,convection otr,Whirlpool,whirlpool 19 cu ft over the range convection m...,convection otr,convection otr,1.000000,...,convection otr,,convection otr,,1.9-cuft over the range convection microwave i...,whirlpool,1.9-cuft over the range convection microwave i...,stainless steel,"[(convection, NN), (otr, NN)]","[(1.9-cuft, JJ), (over, IN), (the, DT), (range..."
6,20,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwave over stove,Whirlpool,whirlpool 19 cu ft over the range convection m...,microwave over stove,microwave over stove,1.000000,...,microwave over stove,,microwave over stove,,1.9-cuft over the range convection microwave i...,whirlpool,1.9-cuft over the range convection microwave i...,stainless steel,"[(microwave, NN), (over, IN), (stove, NN)]","[(1.9-cuft, JJ), (over, IN), (the, DT), (range..."
7,21,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,microwaves,Whirlpool,whirlpool 19 cu ft over the range convection m...,microwaves,microwaves,1.000000,...,microwaves,,microwaves,,1.9-cuft over the range convection microwave i...,whirlpool,1.9-cuft over the range convection microwave i...,stainless steel,"[(microwaves, NNS)]","[(1.9-cuft, JJ), (over, IN), (the, DT), (range..."
8,23,Lithonia Lighting Quantum 2-Light Black LED Em...,100007,2.67,emergency light,Lithonia Lighting,lithonia lighting quantum 2 light black led em...,emergency light,emergency light,1.000000,...,emergency light,,emergency light,,quantum 2 light black led emergency fixture unit,lithonia lighting,quantum 2 light black led emergency fixture unit,,"[(emergency, NN), (light, NN)]","[(quantum, NN), (2, CD), (light, JJ), (black, ..."
9,27,House of Fara 3/4 in. x 3 in.

### Part 6: Attribute Bullets

In [ ]:
t0 = time()

df_attr['product_uid']=df_attr['product_uid'].fillna(0)
df_attr['value']=df_attr['value'].fillna("")
df_attr['name']=df_attr['name'].fillna("")
dict_attr={}
for product_uid in list(set(list(df_attr['product_uid']))):
    dict_attr[int(product_uid)]={'product_uid':int(product_uid),'attribute_bullets':[]}

for i in range(0,len(df_attr['product_uid'])):
    if (i % 100000)==0:
        print "Read",i,"out of", len(df_attr['product_uid']), "rows in attributes.csv in", round((time()-t0)/60,1) ,'minutes'
    if df_attr['name'][i][0:6]=="Bullet":
        dict_attr[int(df_attr['product_uid'][i])]['attribute_bullets'].append(df_attr['value'][i])

if 0 in dict_attr.keys():
    del(dict_attr[0])
                        
for item in dict_attr.keys():
    if len(dict_attr[item]['attribute_bullets'])>0:
        dict_attr[item]['attribute_bullets']=". ".join(dict_attr[item]['attribute_bullets'])
        dict_attr[item]['attribute_bullets']+="."
    else:
        dict_attr[item]['attribute_bullets']=""

                                             
df_attr_bullets=pd.DataFrame(dict_attr).transpose()
df_attr_bullets['attribute_bullets']=df_attr_bullets['attribute_bullets'].map(lambda x: x.replace("..",".").encode('utf-8'))
print 'create attributes bullets time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### Parsing
df_attr_bullets['attribute_bullets_parsed'] = df_attr_bullets['attribute_bullets'].map(lambda x:str_parser(x,add_space_stop_list=[]))
print 'attribute bullets parsing time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### Extracting brands
df_attr_bullets['attribute_bullets_tuple']= df_attr_bullets['attribute_bullets_parsed'].map(lambda x: getremove_brand_or_material_from_str(x,brand_df))
df_attr_bullets['attribute_bullets_parsed_woBrand']= df_attr_bullets['attribute_bullets_tuple'].map(lambda x: x[0])
df_attr_bullets['brands_in_attribute_bullets']= df_attr_bullets['attribute_bullets_tuple'].map(lambda x: x[1])
print 'extract brands from attribute_bullets time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### Extracting  materials
df_attr_bullets['attribute_bullets_tuple']= df_attr_bullets['attribute_bullets_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df))
df_attr_bullets['attribute_bullets_parsed_woBM']= df_attr_bullets['attribute_bullets_tuple'].map(lambda x: x[0])
df_attr_bullets['materials_in_attribute_bullets']= df_attr_bullets['attribute_bullets_tuple'].map(lambda x: x[1])
df_attr_bullets=df_attr_bullets.drop(['attribute_bullets_tuple'],axis=1)
print 'extract materials from attribute_bullets time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### tagging text using NLTK
df_attr_bullets['attribute_bullets_tokens'] =col_tagger(df_attr_bullets['attribute_bullets_parsed_woBM'])
print 'attribute bullets tagging time:',round((time()-t0)/60,1) ,'minutes\n'

### Part 7: Product Descriptions

In [25]:
df_pro_desc = pd.read_csv(DATA_DIR+'/product_descriptions.csv')


### Parsing
df_pro_desc['product_description_parsed'] = df_pro_desc['product_description'].map(lambda x:str_parser(x,add_space_stop_list=add_space_stop_list).encode('utf-8'))
print 'product description parsing time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### Extracting.
df_pro_desc['product_description_tuple']= df_pro_desc['product_description_parsed'].map(lambda x: getremove_brand_or_material_from_str(x,brand_df))
df_pro_desc['product_description_parsed_woBrand']= df_pro_desc['product_description_tuple'].map(lambda x: x[0])
df_pro_desc['brands_in_product_description']= df_pro_desc['product_description_tuple'].map(lambda x: x[1])
print 'extract brands from product_description time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### Extracting  materials
df_pro_desc['product_description_tuple']= df_pro_desc['product_description_parsed_woBrand'].map(lambda x: getremove_brand_or_material_from_str(x,material_df))
df_pro_desc['product_description_parsed_woBM']= df_pro_desc['product_description_tuple'].map(lambda x: x[0])
df_pro_desc['materials_in_product_description']= df_pro_desc['product_description_tuple'].map(lambda x: x[1])
df_pro_desc=df_pro_desc.drop(['product_description_tuple'],axis=1)
print 'extract materials from product_description time:',round((time()-t0)/60,1) ,'minutes\n'
t0 = time()

### ... and tagging text using NLTK
t0 = time()

df_pro_desc['product_description']= df_pro_desc['product_description'].map(lambda x: x.encode('utf-8'))

product description parsing time: 156.6 minutes

extract brands from product_description time: 60.2 minutes

extract materials from product_description time: 3.3 minutes



### Part 8: Stemming

In [32]:
df_pro_desc['product_description_stemmed']=df_pro_desc['product_description_parsed'].map(lambda x:str_stemmer_wo_parser(x))

In [35]:
df_attr_bullets = pd.read_csv(os.getcwd() + '/data/df_attribute_bullets_processed.csv')

In [37]:
df_attr_bullets['attribute_bullets_stemmed']=df_attr_bullets['attribute_bullets_parsed'].map(lambda x:str_stemmer_wo_parser(x))

In [40]:
df_all['search_term_stemmed']=df_all['search_term_parsed'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
df_all['search_term_stemmed_woBM']=df_all['search_term_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
df_all['search_term_stemmed_woBrand']=df_all['search_term_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
df_all['product_title_stemmed']=df_all['product_title_parsed'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
df_all['product_title_stemmed_woBM']=df_all['product_title_parsed_woBM'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))
df_all['product_title_stemmed_woBrand']=df_all['product_title_parsed_woBrand'].map(lambda x:str_stemmer_wo_parser(x,stoplist=stoplist_wo_can))

Save everything into files

In [38]:
df_attr_bullets.to_csv(PROCESSINGTEXT_DIR+"/df_attribute_bullets_processed.csv", index=False,encoding = 'utf-8')
df_pro_desc.to_csv(PROCESSINGTEXT_DIR+"/df_product_descriptions_processed.csv", index=False,encoding = 'utf-8')

In [41]:
df_all.to_csv(PROCESSINGTEXT_DIR+"/df_train_and_test_processed.csv", index=False,encoding = 'utf-8')

df_all[:len(df_train)].to_csv(PROCESSINGTEXT_DIR+"/df_train_processed.csv", index=False,encoding = 'utf-8')
df_all[len(df_train):].to_csv(PROCESSINGTEXT_DIR+"/df_test_processed.csv", index=False,encoding = 'utf-8')